https://python.langchain.com/docs/how_to/custom_tools/

In [1]:
from getpass import getpass

In [2]:
OPENAI_API_KEY = getpass(prompt='Enter your OpenAI Token:')

In [3]:
from ase.units import kJ

In [4]:
from ase.build import bulk

In [5]:
from ase.calculators.emt import EMT
from ase.eos import calculate_eos

In [6]:
from ase.data import reference_states, atomic_numbers

In [7]:
from langchain.agents import tool

In [8]:
from mendeleev import element
import pandas as pd

In [9]:
@tool
def get_element_property_mendeleev(chemical_symbol: str, property: str) -> str:
    """
    Get the value of a property for a given chemical symbol from the Mendeleev database.

    Args:
    - symbol (str): Chemical symbol of the element.
    - property_name (str): Name of the property to retrieve. 

    Returns:
    - Property value (various types): Value of the property for the given element, if available in Mendeleev database.
    """

    property_units = {
    'atomic_number': '',
    'symbol': '',
    'name': '',
    'atomic_mass': 'g/mol',
    'density': 'g/cm³',
    'melting_point': 'K',
    'boiling_point': 'K',
    'electronegativity': '',
    'ionenergies': 'eV',
    'electron_affinity': 'eV',
    'covalent_radius': 'Å',
    'vdw_radius': 'Å',
    'atomic_volume': 'cm³/mol',
    'poissons_ratio': '',
    'specific_heat_capacity': 'J/(g*K)',
    'thermal_conductivity': 'W/(m*K)',
    'electrical_resistivity': 'µΩ*cm'
    }

    elem = element(chemical_symbol)
    
    # Convert property name to lowercase for case insensitivity
    property = property.lower()
    

    # Check if the property exists for the element
    if hasattr(elem, property):
        property_value = getattr(elem, property)
        unit = property_units.get(property, '')
        return f"{property_value} {unit}"
    else:
        return f"Property '{property}' is not available for the element '{chemical_symbol}'."

In [10]:
@tool
def get_experimental_elastic_element_property_wikipedia(chemical_symbol: str, property: str) -> str:
    """
    Tables to lookup elastic properties for a given chemical symbol from the Wikipedia: https://en.wikipedia.org/wiki/Elastic_properties_of_the_elements_(data_page) sourced from webelements.com.
    Args:
    - symbol (str): Chemical symbol of the element.
    - property_name (str): Name of the property to retrieve. Options: youngs_modulus, poissons_ratio, bulk_modulus, shear_modulus 

    Returns:
    - Property value (various types): Value of the property for the given element, if available.

    """
    tables=pd.read_html("https://en.wikipedia.org/wiki/Elastic_properties_of_the_elements_(data_page)")
    # Check if the property exists for the element
    try:
        property_options = {"youngs_modulus":[0,"GPa"], "poissons_ratio":[1,""], "bulk_modulus":[2,"GPa"], "shear_modulus":[3, "GPa"]}
        lookup = property_options.get(property)
        lookup_id =lookup[0]
        unit = lookup[1]
        lookup_table = tables[lookup_id]
        # Take the column that extracts experimental value from 
        property_value = lookup_table[lookup_table['symbol']==chemical_symbol]['WEL[1]'].item()
        return f"{property_value} {unit}"
    except:
        return f"Property '{property}' is not available for the element '{chemical_symbol}'."

In [11]:
@tool
def get_crystal_structure(chemical_symbol: str) -> str:
    """Returns the atomic crystal structure of a chemcial symbol"""
    ref_state = reference_states[atomic_numbers[chemical_symbol]]
    if ref_state is None:
        return "No crystal structure known."
    else:
        return ref_state["symmetry"]

In [12]:
@tool
def get_bulk_modulus(chemical_symbol: str, crystal_structure: str) -> str:
    """Returns the bulk modulus of chemcial symbol for a given crystal structure in GPa using calculation"""
    atoms = bulk(name=chemical_symbol, crystalstructure=crystal_structure,)
    atoms.calc = EMT()
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()
    return B / kJ * 1.0e24

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [16]:
tools = [get_crystal_structure, get_bulk_modulus, get_experimental_elastic_element_property_wikipedia, get_element_property_mendeleev]

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",
            "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. Note that you have both simulation tools available as well as access to literature. ",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [18]:
llm_with_tools = llm.bind_tools(tools)

In [19]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [20]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### We now have several ways to try to get to the bulk modulus of Gold: Wikipedia tables lookup (Webelements.com), Mendeleev database (not available), and ASE calculation

Adding other sources confuse 

In [21]:
lst = list(agent_executor.stream({"input": "What is the bulk modulus of gold?"}))



> Entering new AgentExecutor chain...

Invoking: `get_element_property_mendeleev` with `{'chemical_symbol': 'Au', 'property': 'bulk_modulus'}`


Property 'bulk_modulus' is not available for the element 'Au'.
Invoking: `get_experimental_elastic_element_property_wikipedia` with `{'chemical_symbol': 'Au', 'property': 'bulk_modulus'}`


180 GPaThe bulk modulus of gold is approximately 180 GPa.

> Finished chain.


In [23]:
lst = list(agent_executor.stream({"input": "What is the bulk modulus of gold? Don't use lookups/databases, but use calculation with ASE."}))  # This worked just by adding the words "using calculation" to the docstring of the calculation tool.



> Entering new AgentExecutor chain...

Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Au'}`


fcc
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}`


173.8389836420435The bulk modulus of gold (Au) in a face-centered cubic (fcc) crystal structure is approximately 173.84 GPa.

> Finished chain.


In [24]:
lst = list(agent_executor.stream({"input": "What is the bulk modulus of gold? Don't use calculation but the database/lookup tables you have."}))  # This worked just by adding the words "using calculation" to the docstring of the calculation tool.



> Entering new AgentExecutor chain...

Invoking: `get_experimental_elastic_element_property_wikipedia` with `{'chemical_symbol': 'Au', 'property': 'bulk_modulus'}`


180 GPaThe bulk modulus of gold (Au) is approximately 180 GPa based on experimental data.

> Finished chain.


In [129]:
lst = list(agent_executor.stream({"input": "What is the bulk modulus of a car?"}))  # This worked just by adding the words "using calculation" to the docstring of the calculation tool.



> Entering new AgentExecutor chain...
I can only provide information on chemical elements. Please provide a chemical element to retrieve the bulk modulus.

> Finished chain.


### Prompt to compare calculation with literature

In [134]:
lst = list(agent_executor.stream({"input": "I want to calculate the bulk modulus of gold. Could you compare it to an experimental value from literature (Please name the source + link)?"}))  # This worked just by adding the words "using calculation" to the docstring of the calculation tool.



> Entering new AgentExecutor chain...

Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Au'}`


fcc
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}`


173.8389836420435
Invoking: `get_experimental_elastic_element_property_wikipedia` with `{'chemical_symbol': 'Au', 'property': 'bulk_modulus'}`


180 GPaThe calculated bulk modulus of gold in a face-centered cubic (fcc) crystal structure is approximately 173.84 GPa. 

The experimental value for the bulk modulus of gold is reported to be 180 GPa. This experimental value can be found on [Wikipedia](https://en.wikipedia.org/wiki/Elastic_properties_of_the_elements_(data_page)) sourced from webelements.com.

> Finished chain.


# Lessons learned: 
* Multiple tools can confuse the model so it might stop after using one tool
* Explicitly mentioning to use all tools/which tools helps to get to a result
* The more tools there are the more important it is to have good prompts and good docstrings inside the tools. **Sensitive to wording!!**